이 자료는 Hugging Face Pipeline API를 이용한 챗봇 튜토리얼로 작성되었습니다.  
2025년 9월 14일에 정상 동작을 확인하였습니다.  

1. Hugging Face 회원가입(Sign Up)

    - [Hugging Face 바로가기: https://huggingface.co/](https://huggingface.co/)  
    - Sign Up 따라하기  

2. Hugging Face의 Access Token 생성  

    - 우측상단 (三) 메뉴 -> Settings -> Tokens -> +Create new token
    - 읽기 전용(Read) / 쓰기(Write) / 관리자(Admin) 중 목적에 맞게 발급  
    - 발급된 코드는 다시 볼 수 없기 때문에 별도로 보관  

In [1]:
from huggingface_hub import login
# login(token="hf_xxx_your_token")

3. Transformer 설치  

In [2]:
!pip install --upgrade transformers

4. Colab에서 Hugging Face의 Access Token을 이용하여 로그인(LogIn)  

In [3]:
!hf auth login --token token --add-to-git-credential

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_http.py", line 409, in hf_raise_for_status
    response.raise_for_status()
  File "/usr/local/lib/python3.12/dist-packages/requests/models.py", line 1026, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/whoami-v2

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/huggingface_hub/hf_api.py", line 1782, in whoami
    hf_raise_for_status(r)
  File "/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_http.py", line 482, in hf_raise_for_status
    raise _format(HfHubHTTPError, str(e), response) from e
huggingface_hub.errors.HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/whoami-v2 (Request ID: Root=1-68c6a538-44c846d85452

5. Git credential 설정 하기  

In [4]:
!git config --global credential.helper store

6. Hugging Face 로그인 상태 확인  

In [5]:
!hf auth whoami

gislee


7. 질의응답(Q&A) 모델 불러오기 (bert-large-uncased-whole-word-masking-finetuned-squad)

In [6]:
from transformers import pipeline

qa = pipeline("question-answering")

context = """AI agents are intelligent systems that can perceive the environment,
make decisions, and take actions autonomously. They are increasingly powered by
large language models (LLMs) such as GPT or LLaMA."""

question = "What powers modern AI agents?"
result = qa(question=question, context=context)
print("질문:", question)
print("답변:", result['answer'])

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

Fetching 0 files: 0it [00:00, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 0 files: 0it [00:00, ?it/s]

Device set to use cuda:0


질문: What powers modern AI agents?
답변: large language models


8. Hugging Face Pipeline API를 이용한 챗봇

In [7]:
# ✅ Colab 1) 필수 설치
!pip install -q transformers accelerate sentencepiece gradio

In [8]:
# ✅ Colab 2) 공통 임포트 및 유틸
from transformers import pipeline
import gradio as gr
from typing import List, Tuple

# 대화 히스토리를 문자열로 직렬화하는 간단한 함수
def format_history(history: List[Tuple[str, str]], user_prompt: str, system_prompt: str="You are a helpful assistant.") -> str:
    """
    history: [(user, bot), ...]
    user_prompt: 현재 사용자의 입력
    system_prompt: 시스템 지시
    """
    lines = [f"System: {system_prompt}"]
    for u, b in history:
        lines.append(f"User: {u}")
        lines.append(f"Assistant: {b}")
    lines.append(f"User: {user_prompt}")
    lines.append("Assistant:")
    return "\n".join(lines)

In [9]:
# ✅ Colab 3) 모델 선택 (기본: DialoGPT / 한국어 실험: KoGPT2)
# - DialoGPT: 영어 대화 모델(가볍고 빠름)
# - KoGPT2: 한국어 생성 모델(대화 특화는 아니지만 간단 챗 가능)
USE_KOREAN = False  # 한국어 우선 시범이면 True로

if USE_KOREAN:
    # MODEL_NAME = "skt/kogpt2-base-v2"
    MODEL_NAME = "Llama/Qwen2.5-Instruct"
    TASK = "text-generation"
    GEN_KW = dict(
        max_length=200,
        do_sample=True,
        top_p=0.9,
        temperature=0.8,
        pad_token_id=0  # KoGPT2는 eos/pad 토큰 정의가 특이하므로 0 사용
    )
    system_prompt = "당신은 친절한 한국어 도우미입니다."
else:
    MODEL_NAME = "microsoft/DialoGPT-medium"
    TASK = "text-generation"
    GEN_KW = dict(
        max_length=200,
        do_sample=True,
        top_p=0.9,
        temperature=0.8
    )
    system_prompt = "You are a helpful assistant."

gen = pipeline(TASK, model=MODEL_NAME)
history = []  # [(user, bot), ...]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [10]:
# ✅ Colab 4) 터미널 기반 대화 루프 (원하면 이 셀만 실행)
def chat_once(user_text: str):
    global history
    prompt = format_history(history, user_text, system_prompt=system_prompt)
    out = gen(prompt, **GEN_KW)[0]["generated_text"]
    # 생성된 전체 텍스트에서 마지막 Assistant 발화만 추출(간단 규칙 기반)
    # "Assistant:" 이후를 잘라 사용
    bot_reply = out.split("Assistant:")[-1].strip()
    # 다음 턴을 위해 정리(너무 길면 잘라내기)
    if len(bot_reply) > 800:
        bot_reply = bot_reply[:800]
    history.append((user_text, bot_reply))
    return bot_reply

print("간단 챗봇 준비 완료! chat_once('안녕') 처럼 호출해보세요.")
# 예시:
# reply = chat_once("안녕! 자기소개해줘")
# print("Bot:", reply)

간단 챗봇 준비 완료! chat_once('안녕') 처럼 호출해보세요.


In [13]:
# 예시:
reply = chat_once("안녕! 자기소개해줘")
print("Bot:", reply)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Bot: 


In [14]:
# ✅ Colab 5) Gradio 웹 UI (선택)
with gr.Blocks() as demo:
    gr.Markdown("## 🤗 Hugging Face pipeline 간단 챗봇")
    gr.Markdown(f"- 모델: **{MODEL_NAME}**  |  한국어 옵션: **{USE_KOREAN}**")
    chatbox = gr.Chatbot(height=350)
    txt = gr.Textbox(placeholder="메시지를 입력하고 Enter…")
    clear_btn = gr.Button("대화 리셋")

    def respond(message, chat_history):
        global history
        # 내부 히스토리와 gradio 히스토리 동기화
        # gradio chat_history: [[user, bot], ...]
        history = [(u or "", b or "") for (u, b) in chat_history]
        bot = chat_once(message)
        chat_history.append([message, bot])
        return "", chat_history

    def clear():
        global history
        history = []
        return []

    txt.submit(respond, [txt, chatbox], [txt, chatbox])
    clear_btn.click(clear, inputs=None, outputs=chatbox)

demo.launch(debug=False)

/tmp/ipython-input-2290382659.py:5: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbox = gr.Chatbot(height=350)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b0a2dff8efd0531505.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
